### Read in Data

In [171]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
import seaborn as sns
import math
import sys
%matplotlib inline

census = pd.read_csv('./census_ml.csv')
census.head()

,year,sample,serial,hhwt,cluster,perwt,nchild,nchild_under_5,nsibs,eldch,sex,age,birthyr,race,educ,ed_var_detailed,occ,occ1950,ed_group_h
0,1960,1960 5%,69847,20,1.960000e+12,20,0 children present,No children under age 5,0 siblings,NaN,Male,43,1916,White,"Grade 5, 6, 7, or 8",Grade 8,775,Operative and kindred workers (nec),1
1,1960,1960 5%,2619724,20,1.960030e+12,20,0 children present,No children under age 5,0 siblings,NaN,Male,28,1931,White,Grade 12,Grade 12,290,"Managers, officials, and proprietors (nec)",2
2,1960,1960 5%,2283283,20,1.960020e+12,20,3,No children under age 5,0 siblings,15,Male,36,1923,White,Grade 10,Grade 10,430,Foremen (nec),1
3,1960,1960 5%,2367708,20,1.960020e+12,20,0 children present,No children under age 5,0 siblings,NaN,Male,48,1912,White,Nursery school to grade 4,Grade 3,290,"Managers, officials, and proprietors (nec)",1
4,1960,1960 5%,508827,20,1.960010e+12,20,2,2,0 siblings,2,Male,27,1932,White,Grade 11,Grade 11,394,Salesmen and sales clerks (nec),1


### Clean continuous variables

In [172]:
census.isnull().sum()

year                   0
sample                 0
serial                 0
hhwt                   0
cluster                0
perwt                  0
nchild                 0
nchild_under_5         0
nsibs                  0
eldch              78438
sex                    0
age                    0
birthyr                0
race                   0
educ                   0
ed_var_detailed        0
occ                    0
occ1950                0
ed_group_h             0
dtype: int64

#### Cleaning census['eldch']

In [173]:
census['eldch'].unique()

array([nan, '15', '2', '21', '12', '13', '11', '29', '17', '16', '14',
       'Less than 1 year old', '38', '8', '3', '10', '1', '5', '6', '9',
       '31', '4', '26', '24', '23', '19', '7', '22', '18', '55', '20',
       '27', '53', '35', '34', '37', '28', '36', '25', '60', '33', '52',
       '68', '39', '49', '32', '50', '56', '30', '41', '57', '44', '47',
       '42', '48', '46', '51', '43', '74', '54', '40', '72', '45', '71',
       '59', '66', '62', '58', '65', '63', '61', '73', '69', '64', '70',
       '67', '87'], dtype=object)

In [174]:
census['eldch'] = np.where((census['eldch'] == 'Less than 1 year old'),
                           0, census['eldch'])

In [175]:
census["eldch"].fillna(0, inplace = True) 


In [176]:
census['eldch'][2]
print(type(census['eldch'][2]))

<class 'str'>


In [177]:
for i in range(88):
    census['eldch'] = np.where((census['eldch'] == str(i)),
                           int(i), census['eldch'])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


In [178]:
census['eldch'][2]
#print(type(census['eldch'][2]))

15

#### Cleaning census['nchild]

In [179]:
census['nchild'].unique()

array(['0 children present', '3', '2', '4', '5', '1 child present', '6',
       '7', '9+', '8'], dtype=object)

In [180]:
census['nchild'] = np.where((census['nchild'] == '0 children present'),
                           0, census['nchild'])

In [181]:
census['nchild'] = np.where((census['nchild'] == '1 child present'),
                           1, census['nchild'])

In [182]:
census['nchild'] = np.where((census['nchild'] == '9+'),
                           11, census['nchild'])

In [183]:
for i in range(12):
    census['nchild'] = np.where((census['nchild'] == str(i)),
                           int(i), census['nchild'])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11


#### Cleaning census['nchild_under_5']

In [184]:
census['nchild_under_5'].unique()

array(['No children under age 5', '2', '1 child under age 5', '3', '4',
       '5', '6'], dtype=object)

In [185]:
census['nchild_under_5'] = np.where((census['nchild_under_5'] == 'No children under age 5'),
                           0, census['nchild_under_5'])

census['nchild_under_5'] = np.where((census['nchild_under_5'] == '1 child under age 5'),
                           1, census['nchild_under_5'])

In [186]:
for i in range(7):
    census['nchild_under_5'] = np.where((census['nchild_under_5'] == str(i)),
                           int(i), census['nchild_under_5'])
    print(i)

0
1
2
3
4
5
6


#### Cleaning census['nsibs']

In [187]:
census['nsibs'].unique()

array(['0 siblings', '1 sibling', '4 siblings', '2 siblings',
       '3 siblings', '5 siblings', '6 siblings'], dtype=object)

In [188]:
census['nsibs'] = np.where((census['nsibs'] == '1 sibling'),
                           '1 siblings', census['nsibs'])

In [189]:
for i in range(7):
    census['nsibs'] = np.where((census['nsibs'] == (str(i) + ' siblings')),
                           int(i), census['nsibs'])
    print(i)

0
1
2
3
4
5
6


#### Cleaning census['age']

In [190]:
census['age'].unique()

array(['43', '28', '36', '48', '27', '60', '63', '69', '44', '54', '73',
       '55', '47', '53', '39', '40', '41', '35', '64', '42', '50', '23',
       '38', '57', '46', '31', '76', '70', '32', '22', '25', '45', '30',
       '65', '52', '49', '67', '26', '37', '58', '77', '62', '61', '78',
       '33', '34', '56', '51', '74', '59', '79', '24', '82', '81', '29',
       '20', '66', '68', '72', '71', '75', '85', '21', '93', '19', '80',
       '88', '84', '90 (90+ in 1980 and 1990)', '18', '83', '86', '87',
       '17', '91', '16', '92', '94', '15', '98', '89', '14',
       '100 (100+ in 1960-1970)', '95', '97', '96', '99'], dtype=object)

In [191]:
census['age'] = np.where((census['age'] == "90 (90+ in 1980 and 1990)"),
                           '90', census['age'])

census['age'] = np.where((census['age'] == "100 (100+ in 1960-1970)"),
                           '100', census['age'])

In [192]:
for i in range(101):
    census['age'] = np.where((census['age'] == str(i)),
                           int(i), census['age'])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


#### Drop unnnecessary variables

In [193]:
census.drop(['year', 'sample', 'serial', 'hhwt', 'cluster', 'perwt', 'birthyr', 'occ', 'educ', 'race','ed_var_detailed'], axis=1, inplace=True)

In [194]:
census.head(5)

,nchild,nchild_under_5,nsibs,eldch,sex,age,occ1950,ed_group_h
0,0,0,0,0,Male,43,Operative and kindred workers (nec),1
1,0,0,0,0,Male,28,"Managers, officials, and proprietors (nec)",2
2,3,0,0,15,Male,36,Foremen (nec),1
3,0,0,0,0,Male,48,"Managers, officials, and proprietors (nec)",1
4,2,2,0,2,Male,27,Salesmen and sales clerks (nec),1


In [195]:
census['occ1950'] = np.where((census['occ1950'] == 'N/A (blank)'),
                           float('nan'), census['occ1950'])

census['occ1950'] = np.where((census['occ1950'] == 'Occupation missing/unknown'),
                           float('nan'), census['occ1950'])

In [196]:
census['occ1950'].value_counts()

Managers, officials, and proprietors (nec)          20008
Operative and kindred workers (nec)                 18036
Farmers (owners and tenants)                        11413
Salesmen and sales clerks (nec)                      9966
Laborers (nec)                                       8336
                                                    ...  
Statistics-Professors and instructors                   2
Auto mechanics apprentice                               2
Milliners                                               1
Nurses, student professional                            1
Natural science (nec)-Professors and instructors        1
Name: occ1950, Length: 268, dtype: int64

In [197]:
census = census.dropna()

In [198]:
census.isnull().sum()

nchild            0
nchild_under_5    0
nsibs             0
eldch             0
sex               0
age               0
occ1950           0
ed_group_h        0
dtype: int64

In [199]:
census.head(5)

,nchild,nchild_under_5,nsibs,eldch,sex,age,occ1950,ed_group_h
0,0,0,0,0,Male,43,Operative and kindred workers (nec),1
1,0,0,0,0,Male,28,"Managers, officials, and proprietors (nec)",2
2,3,0,0,15,Male,36,Foremen (nec),1
3,0,0,0,0,Male,48,"Managers, officials, and proprietors (nec)",1
4,2,2,0,2,Male,27,Salesmen and sales clerks (nec),1


In [200]:
census.head(15)

,nchild,nchild_under_5,nsibs,eldch,sex,age,occ1950,ed_group_h
0,0,0,0,0,Male,43,Operative and kindred workers (nec),1
1,0,0,0,0,Male,28,"Managers, officials, and proprietors (nec)",2
2,3,0,0,15,Male,36,Foremen (nec),1
3,0,0,0,0,Male,48,"Managers, officials, and proprietors (nec)",1
4,2,2,0,2,Male,27,Salesmen and sales clerks (nec),1
5,0,0,0,0,Male,60,Purchasing agents and buyers (nec),2
7,0,0,0,0,Male,69,"Lumbermen, raftsmen, and woodchoppers",1
8,0,0,0,0,Male,44,"Managers, officials, and proprietors (nec)",1
9,0,0,1,0,Male,54,Bookkeepers,4
10,0,0,0,0,Male,73,Foremen (nec),1


## TODO: Convert columns from object into int types. 

https://stackoverflow.com/questions/39173813/pandas-convert-dtype-object-to-int


In [201]:
census.dtypes

nchild            object
nchild_under_5    object
nsibs             object
eldch             object
sex               object
age               object
occ1950           object
ed_group_h         int64
dtype: object

In [202]:
census.nchild = census['nchild'].astype('int')
census.nchild_under_5 = census['nchild_under_5'].astype('int')
census.nsibs = census['nsibs'].astype('int')
census.eldch = census['eldch'].astype('int')
census.age = census['age'].astype('int')
#census.sex = census['sex'].astype('int')
census.head(10)

,nchild,nchild_under_5,nsibs,eldch,sex,age,occ1950,ed_group_h
0,0,0,0,0,Male,43,Operative and kindred workers (nec),1
1,0,0,0,0,Male,28,"Managers, officials, and proprietors (nec)",2
2,3,0,0,15,Male,36,Foremen (nec),1
3,0,0,0,0,Male,48,"Managers, officials, and proprietors (nec)",1
4,2,2,0,2,Male,27,Salesmen and sales clerks (nec),1
5,0,0,0,0,Male,60,Purchasing agents and buyers (nec),2
7,0,0,0,0,Male,69,"Lumbermen, raftsmen, and woodchoppers",1
8,0,0,0,0,Male,44,"Managers, officials, and proprietors (nec)",1
9,0,0,1,0,Male,54,Bookkeepers,4
10,0,0,0,0,Male,73,Foremen (nec),1


In [203]:
#saving your data
census.to_csv('cleaned_census.csv')